In [56]:
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import re
import requests
import unidecode
import string
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')


In [57]:
hilary = pd.read_csv('data/hilary_scores.csv')

In [58]:
unique_url = hilary['URL'].unique()

In [68]:
url = unique_url[0]

In [69]:
def tokenize(doc):
    return [snowball.stem(word) for word in word_tokenize(doc.lower())]

tfidfvect = TfidfVectorizer(stop_words='english', tokenizer=tokenize)

In [ ]:
for url in unique_url:
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find_all('p', class_= "story-body-text story-content")
    content = '\n'.join([i.text for i in soup.select('p.story-body-text')])
    content = remove_html_tags(content)
    hilary['Article'] = unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t","")

    documents = hilary[hilary['URL'] == url]['Comment']
    body_text = np.array(body_text)
    documents = np.append(documents.values, body_text)
    
    tfidf_vectorized = tfidfvect.fit_transform(documents).toarray()
    cosine_similarities = []
    cosine_similarities.append(linear_kernel(tfidf_vectorized, tfidf_vectorized))
    
    cosine_similarities[0][len(documents)-1]

In [ ]:
for url in unique_url:
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    body = soup.find_all('p', class_= "story-body-text story-content")
    content = '\n'.join([i.text for i in soup.select('p.story-body-text')])
    body_text = unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t","")
    hilary['Article'] = unidecode.unidecode(content).replace("\n"," ").replace("\'s","").replace("\'t","")

    cond = hilary['URL'] == url
    hilary_url = hilary[cond]
    documents = hilary_url['Comment']
    body_text = np.array(body_text)

    documents = np.append(documents.values, body_text) #body_text is the very last element in the array

    tfidf_vectorized = tfidfvect.fit_transform(documents).toarray()

    similar_to_article = []
    for comment in tfidf_vectorized[:-1]:
        similar_to_article.append(cosine_similarity(comment, tfidf_vectorized[-1])[0, 0])
        
    #similar_to_article = linear_kernel(tfidf_vectorized, tfidf_vectorized)[-1][:-1]
    hilary.loc[cond, 'tfidf_similarity'] = np.array(similar_to_article)

In [82]:
# cosine_similarities[0][49] #similarity between all comments and the article (last element of the array)

array([ 0.28385536,  0.13776017,  0.22468482,  0.19977964,  0.15793206,
        0.10474983,  0.22408546,  0.15809612,  0.16249045,  0.18868696,
        0.17526356,  0.19132463,  0.12223278,  0.090609  ,  0.05319628,
        0.20177948,  0.20985616,  0.25114208,  0.09669662,  0.0664626 ,
        0.27502982,  0.21431282,  0.14668178,  0.16188297,  0.18557274,
        0.15189909,  0.48211165,  0.27101849,  0.10632031,  0.17414081,
        0.11278801,  0.24733688,  0.04340801,  0.08919754,  0.12176521,
        0.12717229,  0.08452504,  0.1488937 ,  0.35451591,  0.03888347,
        0.21992209,  0.20654836,  0.11048562,  0.07464507,  0.11416548,
        0.10454146,  0.34642229,  0.19033833,  0.08244728,  1.        ])

In [97]:
hilary_url.ix[0]

Unnamed: 0                                                             0
Unnamed: 0.1                                                           0
Comment                What Republican wannabe POTUS candidate among ...
URL                    http://takingnote.blogs.nytimes.com/2015/09/03...
Recommendations                                                        0
Locations                                            West Palm Beach, FL
EditorPick                                                         False
userID                                                          33461964
date                                                          1441452450
Sentiment                                                    -0.06691087
Sentiment_sent                                                      -0.2
Sentiment_b                                                            0
Sentiment_b_sent                                                       0
Subjectivity                                       

In [70]:
url

'http://takingnote.blogs.nytimes.com/2015/09/03/donald-trumps-empty-promise-to-the-republican-party/'

In [35]:
#similarity between article and each of the comment
cosine_similarities[0][25]
m = np.array(cosine_similarities[0][25])
m = m[:25]
table["Similarity"] = m
table.head()

array([[ 1.        ,  0.13181099,  0.14152624, ...,  0.11060857,
         0.0944184 ,  0.28385536],
       [ 0.13181099,  1.        ,  0.07040443, ...,  0.04223145,
         0.03672001,  0.13776017],
       [ 0.14152624,  0.07040443,  1.        , ...,  0.06753351,
         0.09677077,  0.22468482],
       ..., 
       [ 0.11060857,  0.04223145,  0.06753351, ...,  1.        ,
         0.05099814,  0.19033833],
       [ 0.0944184 ,  0.03672001,  0.09677077, ...,  0.05099814,
         1.        ,  0.08244728],
       [ 0.28385536,  0.13776017,  0.22468482, ...,  0.19033833,
         0.08244728,  1.        ]])

,Comment,URL,Similarity
313,Obama and the Democrats need to remind the pub...,http://www.nytimes.com/2015/07/16/world/middle...,0.238707
314,Iran realizes that having a bomb is really not...,http://www.nytimes.com/2015/07/16/world/middle...,0.242962
315,After listening on C-Span to the nuclear exper...,http://www.nytimes.com/2015/07/16/world/middle...,0.201110
316,The key question is. How effective are the me...,http://www.nytimes.com/2015/07/16/world/middle...,0.197041
317,"The reality, Iran is and will continue to be a...",http://www.nytimes.com/2015/07/16/world/middle...,0.226393
